In [40]:
!pip install scikit-image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 1.0 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 1.0 MB/s eta 0:00:0000:0100:010m


In [ ]:
!pip install torch torchvision matplotlib

In [26]:
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 1.0 MB/s eta 0:00:0000:0100:010m


In [9]:
!pip install opencv-python pillow numpy matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 1.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 1.0 MB/s eta 0:00:00a 0:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 1.0 MB/s eta 0:00:00a 0:00:01


In [1]:
import os
import cv2
import numpy as np
from PIL import Image, ImageChops, ImageEnhance
import matplotlib.pyplot as plt

def resize_frame(frame, scale_factor=0.5):
    height, width = frame.shape[:2]
    new_size = (int(width * scale_factor), int(height * scale_factor))
    resized_frame = cv2.resize(frame, new_size, interpolation=cv2.INTER_LANCZOS4)
    return resized_frame

def apply_ela(image_path, scale_factor=0.5):
    original = Image.open(image_path)
    original = original.convert("RGB")

    # Resize image
    resized = original.resize((int(original.width * scale_factor), int(original.height * scale_factor)), Image.Resampling.LANCZOS)

    # Convert to numpy arrays
    original_np = np.array(original)
    resized_np = np.array(resized)

    # Calculate error level
    error_level = ImageChops.difference(Image.fromarray(original_np), Image.fromarray(resized_np))

    # Convert error level image to numpy array
    error_level_np = np.array(error_level)

    # Calculate the mean error level value
    mean_error = np.mean(error_level_np)

    return mean_error

def process_videos(video_folder, output_folder):
    # Create the output base directory if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get all video files in the specified folder
    video_files = [os.path.join(video_folder, f) for f in os.listdir(video_folder) if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]

    for video_file in video_files:
        # Create a subfolder for each video
        video_name = os.path.splitext(os.path.basename(video_file))[0]
        video_output_folder = os.path.join(output_folder, video_name)
        if not os.path.exists(video_output_folder):
            os.makedirs(video_output_folder)

        cap = cv2.VideoCapture(video_file)

        if not cap.isOpened():
            print(f"Error: Could not open video file {video_file}")
            continue

        frame_number = 0
        mean_errors = []

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Resize frame
            resized_frame = resize_frame(frame)

            # Save resized frame
            frame_filename = os.path.join(video_output_folder, f"frame_{frame_number}.png")
            cv2.imwrite(frame_filename, resized_frame)

            # Calculate ELA and get mean error level
            mean_error = apply_ela(frame_filename)
            mean_errors.append(mean_error)

            frame_number += 1

        cap.release()
        print(f"Processing complete for {video_file}")

        # Plotting the mean error levels
        plt.figure(figsize=(10, 5))
        plt.plot(mean_errors, label=f'Error Level - {video_name}')
        plt.xlabel('Frame Number')
        plt.ylabel('Mean Error Level')
        plt.title(f'Mean Error Level Across Frames for {video_name}')
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(video_output_folder, 'error_level_plot.png'))
        plt.close()

# Example usage
video_folder = "train_sample_videos"
output_folder = "output_frames"

process_videos(video_folder, output_folder)


Processing complete for train_sample_videos/cdaxixbosp.mp4
Processing complete for train_sample_videos/btiysiskpf.mp4
Processing complete for train_sample_videos/clihsshdkq.mp4
Processing complete for train_sample_videos/alvgwypubw.mp4
Processing complete for train_sample_videos/eqvuznuwsa.mp4
Processing complete for train_sample_videos/eudeqjhdfd.mp4
Processing complete for train_sample_videos/eeyhxisdfh.mp4
Processing complete for train_sample_videos/cizlkenljw.mp4
Processing complete for train_sample_videos/bndybcqhfr.mp4
Processing complete for train_sample_videos/cuzrgrbvil.mp4
Processing complete for train_sample_videos/atyntldecu.mp4
Processing complete for train_sample_videos/bggsurpgpr.mp4
Processing complete for train_sample_videos/eckvhdusax.mp4
Processing complete for train_sample_videos/dvakowbgbt.mp4
Processing complete for train_sample_videos/dqqtjcryjv.mp4
Processing complete for train_sample_videos/djvutyvaio.mp4
Processing complete for train_sample_videos/dzwkmcwkwl.m

In [2]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np


In [3]:
# Load the pre-trained ResNet18 model
model = models.resnet18(pretrained=True)

# Remove the final classification layer to get the feature extraction model
model = torch.nn.Sequential(*list(model.children())[:-1])

# Set the model to evaluation mode
model.eval()


/opt/anaconda3/envs/Implementation/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/Implementation/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [5]:
import os
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np

# Define image transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the pre-trained ResNet18 model
model = models.resnet18(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-1])  # Remove the final classification layer
model.eval()

def extract_features(image_path):
    try:
        # Load and transform the image
        image = Image.open(image_path).convert("RGB")
        image_tensor = transform(image).unsqueeze(0)  # Add batch dimension

        print(f"Processing image: {image_path}")
        print(f"Image tensor shape: {image_tensor.shape}")

        # Pass the image through the model
        with torch.no_grad():
            features = model(image_tensor)
            print(f"Extracted features shape: {features.shape}")

        features = features.squeeze().numpy()
        print(f"Squeezed features shape: {features.shape}")
        return features
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

def process_video_folders(base_folder):
    # Iterate over each folder in the base directory
    for video_folder in os.listdir(base_folder):
        video_folder_path = os.path.join(base_folder, video_folder)
        if not os.path.isdir(video_folder_path):
            continue

        print(f"Processing video folder: {video_folder}")

        # Prepare to store features
        features_list = []

        # Iterate over each file in the video folder
        for file_name in os.listdir(video_folder_path):
            if file_name.startswith("error") and file_name.lower().endswith('.png'):
                ela_image_path = os.path.join(video_folder_path, file_name)

                # Extract features from the ELA image
                features = extract_features(ela_image_path)
                if features is not None and features.size > 0:
                    features_list.append(features)
                else:
                    print(f"Skipped {ela_image_path} due to no features extracted.")

        # Save the features to a numpy file
        features_output_path = os.path.join(video_folder_path, 'features.npy')
        if features_list:
            np.save(features_output_path, np.array(features_list))
            print(f"Features saved for {video_folder}")
        else:
            print(f"No features were extracted for {video_folder}")

# Example usage
base_folder = "output_frames"
process_video_folders(base_folder)


Processing video folder: ahqqqilsxt
Processing image: output_frames/ahqqqilsxt/error_level_plot.png
Image tensor shape: torch.Size([1, 3, 224, 224])
Extracted features shape: torch.Size([1, 512, 1, 1])
Squeezed features shape: (512,)
Features saved for ahqqqilsxt
Processing video folder: djvtbgwdcc
Processing image: output_frames/djvtbgwdcc/error_level_plot.png
Image tensor shape: torch.Size([1, 3, 224, 224])
Extracted features shape: torch.Size([1, 512, 1, 1])
Squeezed features shape: (512,)
Features saved for djvtbgwdcc
Processing video folder: atzdznmder
Processing image: output_frames/atzdznmder/error_level_plot.png
Image tensor shape: torch.Size([1, 3, 224, 224])
Extracted features shape: torch.Size([1, 512, 1, 1])
Squeezed features shape: (512,)
Features saved for atzdznmder
Processing video folder: esyrimvzsa
Processing image: output_frames/esyrimvzsa/error_level_plot.png
Image tensor shape: torch.Size([1, 3, 224, 224])
Extracted features shape: torch.Size([1, 512, 1, 1])
Squeez

In [84]:
np.load('/Users/aniketsaxena/Documents/p/python/project/deepFakeDetection/df1/organized_frames/real/eudeqjhdfd/features.npy', allow_pickle=True)

array([[1.30054009e+00, 1.75150499e-01, 1.47206461e+00, 1.21956062e+00,
        7.80197233e-02, 5.38589209e-02, 4.36189264e-01, 4.47781891e-01,
        1.50521302e+00, 6.22210324e-01, 9.45242960e-03, 2.12246060e+00,
        5.79149663e-01, 5.79591691e-01, 1.41811058e-01, 1.46813661e-01,
        5.45903981e-01, 9.11980748e-01, 1.15721858e+00, 4.55431640e-02,
        3.47043097e-01, 1.53117821e-01, 1.37424842e-01, 1.53564918e+00,
        6.04817450e-01, 2.34549522e-01, 4.98942971e-01, 5.41440785e-01,
        1.03637785e-01, 8.10079947e-02, 1.65779912e+00, 9.80842933e-02,
        1.15696631e-01, 9.21311677e-02, 3.90869111e-01, 1.31433296e+00,
        6.62818849e-02, 9.76481557e-01, 2.83193398e+00, 2.66995788e-01,
        8.43623728e-02, 2.04649150e-01, 3.60843353e-02, 3.89950812e-01,
        5.17459750e-01, 7.21907973e-01, 2.48763800e+00, 0.00000000e+00,
        6.92498028e-01, 9.26984727e-01, 7.98257113e-01, 3.29818702e+00,
        4.48163390e-01, 8.52723420e-02, 5.87730050e-01, 3.254174

In [6]:
import json
import os
import shutil

# Define paths
input_frames_dir = "output_frames"
output_dir = "organized_frames"
metadata_file = "metadata.json"

# Create output directories
os.makedirs(os.path.join(output_dir, "REAL"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "FAKE"), exist_ok=True)

# Read metadata
with open(metadata_file, "r") as f:
    metadata = json.load(f)

# Process each video
for video, info in metadata.items():
    label = info["label"]
    video_name = os.path.splitext(video)[0]  # Remove the .mp4 extension

    # Source and destination paths
    src_path = os.path.join(input_frames_dir, video_name)
    dst_path = os.path.join(output_dir, label, video_name)

    # Check if the source directory exists
    if os.path.exists(src_path):
        # Copy the frame folder to the appropriate category
        shutil.copytree(src_path, dst_path)
        print(f"Copied {video_name} to {label} category")
    else:
        print(f"Warning: Frame folder for {video_name} not found")

print("Organization complete!")

Copied aagfhgtpmv to FAKE category
Copied aapnvogymq to FAKE category
Copied abarnvbtwb to REAL category
Copied abofeumbvv to FAKE category
Copied abqwwspghj to FAKE category
Copied acifjvzvpm to FAKE category
Copied acqfdwsrhi to FAKE category
Copied acxnxvbsxk to FAKE category
Copied acxwigylke to FAKE category
Copied aczrgyricp to FAKE category
Copied adhsbajydo to FAKE category
Copied adohikbdaz to FAKE category
Copied adylbeequz to FAKE category
Copied aelfnikyqj to REAL category
Copied aelzhcnwgf to FAKE category
Copied aettqgevhz to FAKE category
Copied aevrfsexku to FAKE category
Copied afoovlsmtx to REAL category
Copied agdkmztvby to FAKE category
Copied agqphdxmwt to FAKE category
Copied agrmhtjdlk to REAL category
Copied ahbweevwpv to FAKE category
Copied ahdbuwqxit to FAKE category
Copied ahfazfbntc to FAKE category
Copied ahqqqilsxt to REAL category
Copied aipfdnwpoo to FAKE category
Copied ajqslcypsw to REAL category
Copied ajwpjhrbcv to FAKE category
Copied aklqzsddfl to

In [21]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

# Define paths
data_dir = "organized_frames"
categories = ["FAKE", "REAL"]

# Initialize data lists
features = []
labels = []

# Load features and labels
for category in categories:
    category_path = os.path.join(data_dir, category, "")
    label = categories.index(category)
    print(f"Processing {category} videos...")

    for video_folder in tqdm(os.listdir(category_path)):
        video_path = os.path.join(category_path, video_folder)
        features_path = os.path.join(video_path, "features.npy")

        if os.path.exists(features_path):
            video_features = np.load(features_path)
            features.append(video_features)
            labels.append(label)
        else:
            print(f"Warning: features.npy not found in {video_path}")

# Convert lists to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Ensure features and labels are not empty
if len(features) == 0:
    raise ValueError("Features array is empty. Please check the data loading process.")

# Flatten the features if necessary
if len(features.shape) > 2:
    features = features.reshape(features.shape[0], -1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# SVM Model
print("Training SVM model...")
svm_model = SVC(kernel='linear')  # You can experiment with different kernels like 'rbf'
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

# Evaluate SVM
print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm))
print(f"SVM Accuracy: {accuracy_score(y_test, y_pred_svm):.2f}")

# KNN Model
print("Training KNN model...")
knn_model = KNeighborsClassifier(n_neighbors=3)  # You can adjust the number of neighbors
knn_model.fit(X_train, y_train)
y_pred_knn = knn_model.predict(X_test)

# Evaluate KNN
print("KNN Classification Report:")
print(classification_report(y_test, y_pred_knn))
print(f"KNN Accuracy: {accuracy_score(y_test, y_pred_knn):.2f}")

Processing FAKE videos...


100%|██████████| 323/323 [00:00<00:00, 5803.59it/s]


Processing REAL videos...


100%|██████████| 77/77 [00:00<00:00, 5117.92it/s]

Training SVM model...
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.73      0.76        64
           1       0.15      0.19      0.17        16

    accuracy                           0.62        80
   macro avg       0.47      0.46      0.46        80
weighted avg       0.66      0.62      0.64        80

SVM Accuracy: 0.62
Training KNN model...
KNN Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.80      0.79        64
           1       0.13      0.12      0.13        16

    accuracy                           0.66        80
   macro avg       0.46      0.46      0.46        80
weighted avg       0.65      0.66      0.66        80

KNN Accuracy: 0.66


In [24]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

# Define paths
data_dir = "organized_frames"
categories = ["FAKE", "REAL"]

# Initialize data lists
features = []
labels = []

# Load features and labels
for category in categories:
    category_path = os.path.join(data_dir, category, "")
    label = categories.index(category)
    print(f"Processing {category} videos...")

    for video_folder in tqdm(os.listdir(category_path)):
        video_path = os.path.join(category_path, video_folder)
        features_path = os.path.join(video_path, "features.npy")

        if os.path.exists(features_path):
            video_features = np.load(features_path)
            features.append(video_features)
            labels.append(label)
        else:
            print(f"Warning: features.npy not found in {video_path}")

# Convert lists to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Ensure features and labels are not empty
if len(features) == 0:
    raise ValueError("Features array is empty. Please check the data loading process.")

# Flatten the features if necessary
if len(features.shape) > 2:
    features = features.reshape(features.shape[0], -1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# SVM Model
print("Training SVM model...")
svm_model = SVC(kernel='sigmoid')  # You can experiment with different kernels like 'rbf'
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

# Evaluate SVM
print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm))
print(f"SVM Accuracy: {accuracy_score(y_test, y_pred_svm):.2f}")

# KNN Model
print("Training KNN model...")
knn_model = KNeighborsClassifier(n_neighbors=80)  # You can adjust the number of neighbors
knn_model.fit(X_train, y_train)
y_pred_knn = knn_model.predict(X_test)

# Evaluate KNN
print("KNN Classification Report:")
print(classification_report(y_test, y_pred_knn))
print(f"KNN Accuracy: {accuracy_score(y_test, y_pred_knn):.2f}")

Processing FAKE videos...


100%|██████████| 324/324 [00:00<00:00, 12687.47it/s]


Processing REAL videos...


100%|██████████| 78/78 [00:00<00:00, 11884.47it/s]

Training SVM model...
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.80      1.00      0.89        64
           1       0.00      0.00      0.00        16

    accuracy                           0.80        80
   macro avg       0.40      0.50      0.44        80
weighted avg       0.64      0.80      0.71        80

SVM Accuracy: 0.80
Training KNN model...
KNN Classification Report:
              precision    recall  f1-score   support

           0       0.80      1.00      0.89        64
           1       0.00      0.00      0.00        16

    accuracy                           0.80        80
   macro avg       0.40      0.50      0.44        80
weighted avg       0.64      0.80      0.71        80

KNN Accuracy: 0.80



/opt/anaconda3/envs/Implementation/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/Implementation/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/Implementation/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(res

In [20]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

# Define paths
data_dir = "organized_frames"
categories = ["fake", "real"]

# Initialize data lists
features = []
labels = []

# Load features and labels
for category in categories:
    category_path = os.path.join(data_dir, category, "")
    label = categories.index(category)
    print(f"Processing {category} videos...")

    for video_folder in tqdm(os.listdir(category_path)):
        video_path = os.path.join(category_path, video_folder)
        features_path = os.path.join(video_path, "features.npy")

        if os.path.exists(features_path):
            video_features = np.load(features_path)
            features.append(video_features)
            labels.append(label)
        else:
            print(f"Warning: features.npy not found in {video_path}")

# Convert lists to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Ensure features and labels are not empty
if len(features) == 0:
    raise ValueError("Features array is empty. Please check the data loading process.")

# Flatten the features if necessary
if len(features.shape) > 2:
    features = features.reshape(features.shape[0], -1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# SVM Model
print("Training SVM model...")
svm_model = SVC(kernel='linear')  # You can experiment with different kernels like 'rbf'
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

# Evaluate SVM
print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm))
print(f"SVM Accuracy: {accuracy_score(y_test, y_pred_svm):.2f}")

# KNN Model
print("Training KNN model...")
knn_model = KNeighborsClassifier(n_neighbors=3)  # You can adjust the number of neighbors
knn_model.fit(X_train, y_train)
y_pred_knn = knn_model.predict(X_test)

# Evaluate KNN
print("KNN Classification Report:")
print(classification_report(y_test, y_pred_knn))
print(f"KNN Accuracy: {accuracy_score(y_test, y_pred_knn):.2f}")

Processing fake videos...


100%|██████████| 323/323 [00:00<00:00, 3302.30it/s]


Processing real videos...


100%|██████████| 77/77 [00:00<00:00, 3647.92it/s]

Training SVM model...
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.73      0.76        64
           1       0.15      0.19      0.17        16

    accuracy                           0.62        80
   macro avg       0.47      0.46      0.46        80
weighted avg       0.66      0.62      0.64        80

SVM Accuracy: 0.62
Training KNN model...
KNN Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.80      0.79        64
           1       0.13      0.12      0.13        16

    accuracy                           0.66        80
   macro avg       0.46      0.46      0.46        80
weighted avg       0.65      0.66      0.66        80

KNN Accuracy: 0.66


In [25]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

# Define paths
data_dir = "organized_frames"
categories = ["fake", "real"]

# Initialize data lists
features = []
labels = []

# Load features and labels
for category in categories:
    category_path = os.path.join(data_dir, category, "")
    label = categories.index(category)
    print(f"Processing {category} videos...")

    for video_folder in tqdm(os.listdir(category_path)):
        video_path = os.path.join(category_path, video_folder)
        features_path = os.path.join(video_path, "features.npy")

        if os.path.exists(features_path):
            video_features = np.load(features_path)
            features.append(video_features)
            labels.append(label)
        else:
            print(f"Warning: features.npy not found in {video_path}")

# Convert lists to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Ensure features and labels are not empty
if len(features) == 0:
    raise ValueError("Features array is empty. Please check the data loading process.")

# Flatten the features if necessary
if len(features.shape) > 2:
    features = features.reshape(features.shape[0], -1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# SVM Model
print("Training SVM model...")
svm_model = SVC(kernel='sigmoid')  # You can experiment with different kernels like 'rbf'
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

# Evaluate SVM
print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm))
print(f"SVM Accuracy: {accuracy_score(y_test, y_pred_svm):.2f}")

# KNN Model
print("Training KNN model...")
knn_model = KNeighborsClassifier(n_neighbors=4)  # You can adjust the number of neighbors
knn_model.fit(X_train, y_train)
y_pred_knn = knn_model.predict(X_test)

# Evaluate KNN
print("KNN Classification Report:")
print(classification_report(y_test, y_pred_knn))
print(f"KNN Accuracy: {accuracy_score(y_test, y_pred_knn):.2f}")

Processing fake videos...


100%|██████████| 324/324 [00:00<00:00, 1760.39it/s]


Processing real videos...


  0%|          | 0/78 [00:00<?, ?it/s]

100%|██████████| 78/78 [00:00<00:00, 1895.51it/s]

Training SVM model...
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.80      1.00      0.89        64
           1       0.00      0.00      0.00        16

    accuracy                           0.80        80
   macro avg       0.40      0.50      0.44        80
weighted avg       0.64      0.80      0.71        80

SVM Accuracy: 0.80
Training KNN model...
KNN Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.98      0.89        64
           1       0.67      0.12      0.21        16

    accuracy                           0.81        80
   macro avg       0.74      0.55      0.55        80
weighted avg       0.79      0.81      0.76        80

KNN Accuracy: 0.81



/opt/anaconda3/envs/Implementation/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/Implementation/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/Implementation/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(res

In [18]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

# Define paths
data_dir = "organized_frames"
categories = ["fake", "real"]

# Initialize data lists
features = []
labels = []

# Load features and labels
for category in categories:
    category_path = os.path.join(data_dir, category, "")
    label = categories.index(category)
    print(f"Processing {category} videos...")

    for video_folder in tqdm(os.listdir(category_path)):
        video_path = os.path.join(category_path, video_folder)
        features_path = os.path.join(video_path, "features.npy")

        if os.path.exists(features_path):
            video_features = np.load(features_path)
            features.append(video_features)
            labels.append(label)
        else:
            print(f"Warning: features.npy not found in {video_path}")

# Convert lists to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Ensure features and labels are not empty
if len(features) == 0:
    raise ValueError("Features array is empty. Please check the data loading process.")

# Flatten the features if necessary
if len(features.shape) > 2:
    features = features.reshape(features.shape[0], -1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# SVM Model
print("Training SVM model...")
svm_model = SVC(kernel='poly')  # You can experiment with different kernels like 'rbf'
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

# Evaluate SVM
print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm))
print(f"SVM Accuracy: {accuracy_score(y_test, y_pred_svm):.2f}")

# KNN Model
print("Training KNN model...")
knn_model = KNeighborsClassifier(n_neighbors=10)  # You can adjust the number of neighbors
knn_model.fit(X_train, y_train)
y_pred_knn = knn_model.predict(X_test)

# Evaluate KNN
print("KNN Classification Report:")
print(classification_report(y_test, y_pred_knn))
print(f"KNN Accuracy: {accuracy_score(y_test, y_pred_knn):.2f}")

Processing fake videos...


100%|██████████| 323/323 [00:00<00:00, 12719.08it/s]


Processing real videos...


100%|██████████| 77/77 [00:00<00:00, 11563.24it/s]

Training SVM model...
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.80      1.00      0.89        64
           1       0.00      0.00      0.00        16

    accuracy                           0.80        80
   macro avg       0.40      0.50      0.44        80
weighted avg       0.64      0.80      0.71        80

SVM Accuracy: 0.80
Training KNN model...
KNN Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.98      0.89        64
           1       0.67      0.12      0.21        16

    accuracy                           0.81        80
   macro avg       0.74      0.55      0.55        80
weighted avg       0.79      0.81      0.76        80

KNN Accuracy: 0.81



/opt/anaconda3/envs/Implementation/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/Implementation/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/Implementation/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(res